<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: **SUPER IMPORTANTE - notebooks sin nombre no serán revisados**

- Nombre de alumno 1: Sergio Rehbein
- Nombre de alumno 2: Matías Cornejo

\### **Link de repositorio de GitHub Matias:** https://github.com/s-kill/MDS7202

\### **Link de repositorio de GitHub Sergio:** https://github.com/sergiorehbein/MDS7201---Proyecto-de-Ciencia-de-Datos


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** `http://....`

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Importamos librerias útiles

In [4]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime

df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

<ipython-input-1-a738af29f7fb>:6: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['date'] = pd.to_datetime(df['date'])


,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [25]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import FunctionTransformer, OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor
from sklearn.metrics import mean_absolute_error
from xgboost import XGBRegressor
import joblib


data = df.copy()

# División de los datos en conjuntos de entrenamiento, validación y prueba
train_data, temp_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(temp_data, test_size=1/3, random_state=42)

# Función para extraer día, mes y año de la fecha
def extract_date_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'], format='%d/%m/%y')
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df.drop(columns=['date','id'])

# Transformer para la extracción de características de la fecha
date_transformer = FunctionTransformer(extract_date_features)

# Aplicar la transformación a los datos de entrenamiento y validación
transformed_train_data = date_transformer.fit_transform(train_data)
transformed_val_data = date_transformer.transform(val_data)

# Separar características y variable objetivo
X_train = transformed_train_data.drop(columns=['quantity'])
y_train = transformed_train_data['quantity']

X_val = transformed_val_data.drop(columns=['quantity'])
y_val = transformed_val_data['quantity']

# Definir el column transformer
numeric_features = ['lat', 'long', 'pop', 'price']
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
    ])

# Definir el pipeline con DummyRegressor
pipeline_dummy = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

# Definir el pipeline con XGBRegressor
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

# Entrenar el pipeline con DummyRegressor
pipeline_dummy.fit(X_train, y_train)

# Predicciones y cálculo del MAE con DummyRegressor
y_val_pred_dummy = pipeline_dummy.predict(X_val)
mae_dummy = mean_absolute_error(y_val, y_val_pred_dummy)

# Entrenar el pipeline con XGBRegressor
pipeline_xgb.fit(X_train, y_train)

# Predicciones y cálculo del MAE con XGBRegressor
y_val_pred_xgb = pipeline_xgb.predict(X_val)
mae_xgb = mean_absolute_error(y_val, y_val_pred_xgb)

# Guardar ambos modelos en archivos .pkl
joblib.dump(pipeline_dummy, 'dummy_regressor_pipeline.pkl')
joblib.dump(pipeline_xgb, 'xgb_regressor_pipeline.pkl')

# Imprimir los resultados
print(f'MAE DummyRegressor: {mae_dummy}')
print(f'MAE XGBRegressor: {mae_xgb}')

MAE DummyRegressor: 13298.497767341096
MAE XGBRegressor: 2433.320936196607


## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




In [49]:
X_train.columns

Index(['city', 'lat', 'long', 'pop', 'shop', 'brand', 'container', 'capacity',
       'price', 'day', 'month', 'year'],
      dtype='object')

In [51]:
import xgboost as xgb
# Ajustamos el preprocesador para obtener los nombres de las características procesadas
preprocessor.fit(X_train)
X_train_processed = preprocessor.transform(X_train)
X_val_processed = preprocessor.transform(X_val)

# Crear DMatrix para XGBoost
dtrain = xgb.DMatrix(X_train_processed, label=y_train)
dval = xgb.DMatrix(X_val_processed, label=y_val)

# Definir parámetros con restricciones de monotonía
# La restricción se aplica en la posición correspondiente a 'price' después de OneHotEncoding
params = {
    'objective': 'reg:squarederror',
    'monotone_constraints': '(0,0,0,0,0,0,0,0,-1,0,0,0)'  # Ajustar según el número de características
}

# Definir el conjunto de evaluación
evallist = [(dtrain, 'train'), (dval, 'eval')]

# Entrenar el modelo con restricciones de monotonía
model_with_constraints = xgb.train(params, dtrain, num_boost_round=1000, evals=evallist, early_stopping_rounds=10)

# Predicciones y cálculo del MAE
y_val_pred_xgb_monotonic = model_with_constraints.predict(dval)
mae_xgb_monotonic = mean_absolute_error(y_val, y_val_pred_xgb_monotonic)

# Guardar el modelo
joblib.dump(model_with_constraints, 'xgb_regressor_monotonic_model.pkl')

# Imprimir los resultados
print(f'MAE XGBRegressor with Monotonic Constraint: {mae_xgb_monotonic}')

[0]	train-rmse:14358.74031	eval-rmse:14425.54552
[1]	train-rmse:12086.40610	eval-rmse:12310.91967
[2]	train-rmse:10363.16486	eval-rmse:10705.48184
[3]	train-rmse:9015.11646	eval-rmse:9497.48118
[4]	train-rmse:8125.69084	eval-rmse:8653.92247
[5]	train-rmse:7309.49944	eval-rmse:7913.85200
[6]	train-rmse:6742.70352	eval-rmse:7385.17539
[7]	train-rmse:6116.11076	eval-rmse:6818.22918
[8]	train-rmse:5762.69212	eval-rmse:6503.54446
[9]	train-rmse:5319.56047	eval-rmse:6100.98820
[10]	train-rmse:5020.93902	eval-rmse:5864.68814
[11]	train-rmse:4792.98022	eval-rmse:5656.41176
[12]	train-rmse:4519.17089	eval-rmse:5405.79187
[13]	train-rmse:4369.70338	eval-rmse:5293.33037
[14]	train-rmse:4118.68878	eval-rmse:5067.56430
[15]	train-rmse:3952.11328	eval-rmse:4934.09139
[16]	train-rmse:3829.15266	eval-rmse:4826.21858
[17]	train-rmse:3726.88112	eval-rmse:4782.57366
[18]	train-rmse:3632.17398	eval-rmse:4722.63810
[19]	train-rmse:3494.02078	eval-rmse:4601.30130
[20]	train-rmse:3440.97291	eval-rmse:4564.87

Aplicar Monotonic Contraint baja el MAE de 2433.32 a 2296.99

## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [53]:
from optuna.samplers import TPESampler
import optuna
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split, cross_val_score
def objective(trial):
    # Optimizar hiperparámetros del preprocesador
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    params = {
        'objective': 'reg:squarederror',
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'monotone_constraints':'(0,0,0,0,0,0,0,0,-1,0,0,0)'
    }

    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    categorical_transformer.set_params(min_frequency=min_frequency)
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])

    # Entrenar el modelo con restricciones de monotonía
    model = XGBRegressor(seed=42, **params)

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Predicciones y cálculo del MAE
    pipeline.fit(X_train, y_train)
    y_val_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_val_pred)

    return mae

# Fijar la semilla para reproducibilidad
optuna.logging.set_verbosity(optuna.logging.WARNING) #Quitar Log
sampler = TPESampler(seed=42)

# Crear y ejecutar el estudio
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100, timeout=300) #Fijado a 100 trials o 5 min


In [54]:
# Reportar los resultados
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Number of finished trials:  100
Best trial:
  Value:  1961.6943135047263
  Params: 
    min_frequency: 0.002924968027274369
    learning_rate: 0.058141548044030086
    n_estimators: 998
    max_depth: 8
    max_leaves: 78
    min_child_weight: 4
    reg_alpha: 0.5899725982095636
    reg_lambda: 0.3757980010998163


In [61]:
#Mejore parametros
best_params = study.best_params
#Mejor modelo
# Crear y ajustar el modelo final
preprocessor = ColumnTransformer(
transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', min_frequency=best_params['min_frequency']), categorical_features)
])

# Entrenar el modelo con mejores parametros
model = XGBRegressor(
    seed=42,
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    max_leaves=best_params['max_leaves'],
    min_child_weight=best_params['min_child_weight'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])

# Entrenamiento y Dump
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, 'best_model.pkl')

['preprocessor.pkl']

Explicación:
1. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?:

Hiperparámetros del Modelo:

* min_frequency: Este parámetro del OneHotEncoder controla la frecuencia mínima de las categorías para ser incluidas en la codificación. Un valor más alto reduce la dimensionalidad eliminando categorías raras. Rango (0.0, 1.0) es razonable, ya que cubre desde no filtrar categorías hasta eliminar las más raras.
* learning_rate: Tasa de aprendizaje que controla el tamaño del paso en cada iteración. Valores más bajos permiten que el modelo aprenda de manera más precisa pero requieren más iteraciones. Rango (0.001, 0.1) es adecuado para evitar tanto el subajuste como el sobreajuste.
* n_estimators: Número de árboles en el modelo. Más árboles generalmente mejoran el ajuste pero pueden aumentar el riesgo de sobreajuste. Rango (50, 1000) es amplio para explorar tanto modelos más simples como más complejos.
* max_depth: Profundidad máxima de los árboles. Controla la complejidad del modelo. Valores más altos pueden capturar relaciones más complejas pero también pueden llevar al sobreajuste. Rango (3, 10) es adecuado para encontrar un equilibrio entre complejidad y generalización.
* max_leaves: Número máximo de hojas en los árboles. Similar a max_depth, controla la complejidad del modelo. Rango (0, 100) es razonable para explorar desde modelos muy simples hasta muy complejos.
* min_child_weight: Peso mínimo de la suma de instancias necesarias en una hoja hija. Ayuda a evitar el sobreajuste. Rango (1, 5) es adecuado para controlar la división de hojas en el árbol.
* reg_alpha: Término de regularización L1 que penaliza las características menos importantes, ayudando a evitar el sobreajuste. Rango (0, 1) es adecuado para explorar desde ninguna penalización hasta una penalización fuerte.
* reg_lambda: Término de regularización L2 que penaliza las características de manera diferente a reg_alpha. Rango (0, 1) es adecuado para explorar diferentes niveles de penalización.
*monotone_constraints: Restricciones de monotonía para garantizar que el modelo respete una relación monótona entre las características y la variable objetivo. En este caso, garantiza que un aumento en el precio (price) resulte en una disminución en la cantidad (quantity).

2. Reportar el número de trials, el MAE y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?:

Número de Trials, MAE y Mejores Hiperparámetros Encontrados:

* Número de trials: 100
* MAE (Mean Absolute Error): 1961.69
Mejores Hiperparámetros:
* min_frequency: 0.002924968027274369
* learning_rate: 0.058141548044030086
* n_estimators: 998
* max_depth: 8
* max_leaves: 78
* min_child_weight: 4
* reg_alpha: 0.5899725982095636
* reg_lambda: 0.3757980010998163

De lo anterior podemos notar que:
* MAE con Hiperparámetros Optimizados (Optuna): 1961.69
* MAE Anterior (sin Optuna): 2296.988080173751

El MAE ha mejorado significativamente con la optimización de hiperparámetros utilizando Optuna. Esta mejora se debe a que el proceso de optimización bayesiana explora sistemáticamente el espacio de hiperparámetros para encontrar la combinación que minimiza el error de predicción

## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [64]:
!pip install optuna-integration

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.4/93.4 kB 2.2 MB/s eta 0:00:00


In [75]:
from optuna.samplers import TPESampler
import optuna
from xgboost import XGBRegressor
def objective(trial):
    # Optimizar hiperparámetros del preprocesador
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    params = {
        'objective': 'reg:squarederror',
        "eval_metric": "mae",
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1),
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'max_leaves': trial.suggest_int('max_leaves', 0, 100),
        'min_child_weight': trial.suggest_int('min_child_weight', 1, 5),
        'reg_alpha': trial.suggest_float('reg_alpha', 0, 1),
        'reg_lambda': trial.suggest_float('reg_lambda', 0, 1),
        'monotone_constraints':'(0,0,0,0,0,0,0,0,-1,0,0,0)'
    }

    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)
    categorical_transformer = OneHotEncoder(handle_unknown='ignore')
    categorical_transformer.set_params(min_frequency=min_frequency)
    preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', categorical_transformer, categorical_features)
    ])
    # Entrenar el modelo con prunning
    pruning_callback = optuna.integration.XGBoostPruningCallback(
        trial, observation_key="validation_0-mae"
    )
    # Entrenar el modelo con restricciones de monotonía
    model = XGBRegressor(seed=42, **params)

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])
    X_train_transformed = preprocessor.fit_transform(X_train)
    X_val_transformed = preprocessor.transform(X_val)
    # Predicciones y cálculo del MAE
    pipeline.fit(X_train, y_train, regressor__eval_set=[(X_val_transformed, y_val)], regressor__early_stopping_rounds=10, regressor__callbacks=[pruning_callback])
    y_val_pred = pipeline.predict(X_val)
    mae = mean_absolute_error(y_val, y_val_pred)

    return mae

# Fijar la semilla para reproducibilidad
optuna.logging.set_verbosity(optuna.logging.WARNING) #Quitar Log
sampler = TPESampler(seed=42)

# Crear y ejecutar el estudio
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, n_trials=100, timeout=300, show_progress_bar = True) #Fijado a 100 trials o 5 min

  0%|          | 0/100 [00:00<?, ?it/s]

[0]	validation_0-mae:12592.86576
[1]	validation_0-mae:11998.08750
[2]	validation_0-mae:11492.05480
[3]	validation_0-mae:11048.62646
[4]	validation_0-mae:10665.51945
[5]	validation_0-mae:10346.54007
[6]	validation_0-mae:10084.89470


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[7]	validation_0-mae:9869.63969
[8]	validation_0-mae:9673.69848
[9]	validation_0-mae:9501.85683
[10]	validation_0-mae:9329.10604
[11]	validation_0-mae:9199.70781
[12]	validation_0-mae:9091.05398
[13]	validation_0-mae:8982.87844
[14]	validation_0-mae:8891.23863
[15]	validation_0-mae:8811.53678
[16]	validation_0-mae:8733.13107
[17]	validation_0-mae:8673.36623
[18]	validation_0-mae:8627.67944
[19]	validation_0-mae:8579.37769
[20]	validation_0-mae:8546.48327
[21]	validation_0-mae:8519.31996
[22]	validation_0-mae:8492.86745
[23]	validation_0-mae:8464.73626
[24]	validation_0-mae:8440.39767
[25]	validation_0-mae:8427.25415
[26]	validation_0-mae:8409.20505
[27]	validation_0-mae:8397.23210
[28]	validation_0-mae:8384.48886
[29]	validation_0-mae:8361.96410
[30]	validation_0-mae:8351.30970
[31]	validation_0-mae:8349.69375
[32]	validation_0-mae:8336.44146
[33]	validation_0-mae:8339.38079
[34]	validation_0-mae:8327.85994
[35]	validation_0-mae:8313.80361
[36]	validation_0-mae:8308.19398
[37]	validati

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


...

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[11]	validation_0-mae:6866.73934
[12]	validation_0-mae:6602.01846
[13]	validation_0-mae:6355.68504
[14]	validation_0-mae:6128.37029
[15]	validation_0-mae:5912.04096
[16]	validation_0-mae:5685.45609
[17]	validation_0-mae:5486.88020
[18]	validation_0-mae:5283.46066
[19]	validation_0-mae:5114.88065
[20]	validation_0-mae:4950.24370
[21]	validation_0-mae:4800.73667
[22]	validation_0-mae:4652.52359
[23]	validation_0-mae:4543.05803
[24]	validation_0-mae:4416.03949
[25]	validation_0-mae:4309.94492
[26]	validation_0-mae:4194.22536
[27]	validation_0-mae:4100.36697
[28]	validation_0-mae:4005.36749
[29]	validation_0-mae:3922.10753
[30]	validation_0-mae:3829.23859
[31]	validation_0-mae:3739.91159
[32]	validation_0-mae:3659.84532
[33]	validation_0-mae:3583.50871
[34]	validation_0-mae:3518.31042
[35]	validation_0-mae:3464.05930
[36]	validation_0-mae:3407.36116
[37]	validation_0-mae:3367.15156
[38]	validation_0-mae:3315.02669
[39]	validation_0-mae:3262.32350
[40]	validation_0-mae:3216.26805
[41]	valid

/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


[0]	validation_0-mae:12462.00928
[1]	validation_0-mae:11738.89166
[2]	validation_0-mae:11071.27787
[3]	validation_0-mae:10475.06817
[4]	validation_0-mae:9944.69952
[5]	validation_0-mae:9470.68291
[6]	validation_0-mae:9058.77919
[7]	validation_0-mae:8700.26669
[8]	validation_0-mae:8379.28002
[9]	validation_0-mae:8078.81250


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `early_stopping_rounds` in `fit` method is deprecated for better compatibility with scikit-learn, use `early_stopping_rounds` in constructor or`set_params` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:889: UserWarning: `callbacks` in `fit` method is deprecated for better compatibility with scikit-learn, use `callbacks` in constructor or`set_params` instead.
  warnings.warn(


Prunning permitirá "podar" todos aquellos intentos que tienden a disminuir el desempeño final del modelo

In [76]:
# Reportar los resultados
print("Number of finished trials: ", len(study.trials))
print("Best trial:")
trial = study.best_trial

print("  Value: ", trial.value)
print("  Params: ")
for key, value in trial.params.items():
    print(f"    {key}: {value}")

Number of finished trials:  100
Best trial:
  Value:  1972.1627931652415
  Params: 
    min_frequency: 0.06682216541029647
    learning_rate: 0.0916109108265852
    n_estimators: 648
    max_depth: 9
    max_leaves: 90
    min_child_weight: 3
    reg_alpha: 0.3613749021550724
    reg_lambda: 0.6886578311868585


In [79]:
#Mejore parametros
best_params = study.best_params
#Mejor modelo
# Crear y ajustar el modelo final
preprocessor = ColumnTransformer(
transformers=[
    ('num', StandardScaler(), numeric_features),
    ('cat', OneHotEncoder(handle_unknown='ignore', min_frequency=best_params['min_frequency']), categorical_features)
])

# Entrenar el modelo con mejores parametros
model = XGBRegressor(
    seed=42,
    learning_rate=best_params['learning_rate'],
    n_estimators=best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    max_leaves=best_params['max_leaves'],
    min_child_weight=best_params['min_child_weight'],
    reg_alpha=best_params['reg_alpha'],
    reg_lambda=best_params['reg_lambda'])

pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', model)
])

# Entrenamiento y Dump
pipeline.fit(X_train, y_train)
joblib.dump(pipeline, 'best_model.pkl')

['best_model.pkl']

* MAE con Hiperparámetros Optimizados sin prunning (Optuna): 1961.69
* MAE con Hiperparámetros Optimizados con prunning (Optuna): 1972.16


El MAE subió levemente comparado al metodo sin prunning, esto no es algo extraño, ya que el método de prunning no cambia la optimización en sí, sino que "poda" pruebas o trials menos prometedoras, acelerando el proceso de actualización. Esta leve diferencia puede estar dada por una razón más probabilistica.

## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [77]:
# Crear las visualizaciones
fig1 = optuna.visualization.plot_optimization_history(study)
fig2 = optuna.visualization.plot_parallel_coordinate(study)
fig3 = optuna.visualization.plot_param_importances(study)

In [78]:
# Mostrar las visualizaciones
fig1.show()
fig2.show()
fig3.show()

*   Del primer gráfico, se observa que las mejoras son a partir del trial = 4. En este punto hay una caida significativa en el valor objetivo.

*   Del segundo gráfico:
  *   Los valores bajos de learning_rate (alrededor de 0.05) tienden a estar asociados con menores valores de error (MAE).

  *   Los valores más altos de max_depth (alrededor de 8-10) están asociados con mejores resultados.
  *   min_frequency muestra que valores más bajos tienden a estar asociados con un menor MAE.
  *   La cantidad de estimadores (n_estimators) no tiene una tendencia clara, pero los valores intermedios a altos (alrededor de 500-800) parecen estar asociados con mejores resultados.

* Del Tercer grafico:

  * min_frequency: Es el hiperparámetro más importante, con una importancia de 0.50. Esto indica que ajustar la frecuencia mínima de las categorías tiene un gran impacto en el rendimiento del modelo.
  * learning_rate: Tiene una importancia de 0.18, lo que sugiere que el ajuste de la tasa de aprendizaje es crucial para el rendimiento del modelo.
  * reg_alpha y max_depth: Ambos tienen una importancia significativa (0.12 y 0.11, respectivamente), lo que indica que la regularización L1 y la profundidad máxima de los árboles son importantes para el modelo.
  * n_estimators y reg_lambda: También tienen una cierta importancia (0.05 y 0.03, respectivamente), aunque menor en comparación con los otros hiperparámetros.







## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

Resultados:
*   XGBRegressor_Baseline:	2433.32
*   XGBRegressor_Monotonic_Constraint:	2296.99
*   XGBRegressor_Optimizado_Sin_Pruners:	1961.69
*   XGBRegressor_Optimizado_Con_Pruners:	1972.16

El modelo que obtiene el mejor rendimiento es el XGBRegressor_Optimizado_Sin_Pruners con un MAE de 1961.69.
Aunque el modelo con pruning también tiene un rendimiento cercano, no supera al modelo optimizado sin pruning.





In [80]:
# Cargar el mejor modelo
best_model = joblib.load('best_model.pkl')

In [89]:
transformed_test_data = date_transformer.transform(test_data)
X_test = transformed_test_data.drop(columns=['quantity'])
y_test = transformed_test_data['quantity']
# Predecir sobre el conjunto de test
y_test_pred = best_model.predict(X_test)
y_val_pred = best_model.predict(X_val)

# Calcular el MAE en el conjunto de test
mae_eval = mean_absolute_error(y_val, y_val_pred)
mae_test = mean_absolute_error(y_test, y_test_pred)

print(f"MAE en el conjunto de validación: {mae_eval}")
print(f"MAE en el conjunto de test: {mae_test}")

MAE en el conjunto de validación: 1981.524438787674
MAE en el conjunto de test: 1990.437329494282


Las métricas para el conjunto de validación y test son bastante cercanas, pero hay una ligera diferencia.

Aunque el conjunto de validación y el conjunto de test son muestras del mismo conjunto de datos original, pueden tener distribuciones ligeramente diferentes, lo que puede llevar a diferencias en las métricas.

Por otro lado, el proceso de entrenamiento de XGBoost incluye elementos estocásticos como la inicialización de pesos y el muestreo de datos. Aunque estos efectos suelen ser pequeños, pueden contribuir a ligeras diferencias en el rendimiento entre conjuntos.

En conclusión, las diferencias observadas entre las métricas de validación y test son normales y esperadas hasta cierto punto, debido a los factores mencionados anteriormente. La proximidad de las métricas sugiere que el modelo generaliza bien a datos no vistos, aunque siempre existe un margen para mejoras adicionales.

# Conclusión
El proceso de optimización de modelos XGBoost, que incluyó restricciones monotónicas, ajuste de hiperparámetros con Optuna y técnicas de pruning, mejoró significativamente el rendimiento del modelo. XGBRegressor optimizado con pruning, obtuvo un MAE de 1981.52 en validación y 1990.44 en test, demostrando una buena capacidad de generalización. Las visualizaciones destacaron la importancia del hiperparámetro min_frequency. En resumen, el enfoque utilizado resultó efectivo para optimizar modelos de predicción en tareas de regresión.

---------------------


Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>